In [2]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
df = pd.read_hdf('data/v0/train.h5',key='table')

In [4]:
arr = df.iloc[:,:-6].to_numpy()
arr.shape

(1211000, 800)

In [5]:
arr = arr.reshape(-1,4)
arr.shape

(242200000, 4)

In [6]:
par_df= pd.DataFrame(data=arr, columns=['E','PX','PY','PZ'])

tempArr =np.repeat(np.array([i for i in range(df.shape[0])]),200)
par_df['j_index'] = tempArr

par_df['j_t'] = np.repeat(df['is_signal_new'].values,200)
par_df.info

<bound method DataFrame.info of                     E          PX          PY          PZ  j_index  j_t
0          474.071136 -250.347031 -223.651962 -334.738098        0    0
1          103.236237  -48.866222  -56.790775  -71.025490        0    0
2          105.255569  -55.415001  -49.968887  -74.236267        0    0
3           40.176777  -21.760696  -18.717611  -28.112215        0    0
4           22.428583  -11.835756  -10.374107  -15.979177        0    0
...               ...         ...         ...         ...      ...  ...
242199995    0.000000    0.000000    0.000000    0.000000  1210999    1
242199996    0.000000    0.000000    0.000000    0.000000  1210999    1
242199997    0.000000    0.000000    0.000000    0.000000  1210999    1
242199998    0.000000    0.000000    0.000000    0.000000  1210999    1
242199999    0.000000    0.000000    0.000000    0.000000  1210999    1

[242200000 rows x 6 columns]>

In [7]:
par_df = par_df.loc[~(par_df.iloc[:,:-2]==0).all(axis=1),:]
par_df = par_df.reset_index(drop=True)
del df
par_df.shape

(59398621, 6)

In [8]:
# jet level features
jet_gp = par_df.groupby(['j_index'])

j_px = jet_gp['PX'].sum()
j_py = jet_gp['PY'].sum()
j_pz = jet_gp['PZ'].sum()
j_e  = jet_gp['E'].sum()

j_pt = (j_px**2+j_py**2)**.5
j_mass = (j_e**2-j_pt**2-j_pz**2)**.5
j_phi = np.arcsin(j_py/j_pt)
j_eta = np.arcsinh(j_pz/j_pt)

# j_theta = np.arctan2(j_py, j_px)

D:\Softwares\Anacoda\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in arcsin
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [14]:
temp = (j_px[0]*par_df.loc[0,'PX']+j_py[0]*par_df.loc[0,'PY']+j_pz[0]*par_df.loc[0,'PZ'])/((j_px[0]**2+j_py[0]**2+j_pz[0]**2)**.5)/((par_df.loc[0,'PX']**2+par_df.loc[0,'PY']**2+par_df.loc[0,'PZ']**2)**.5)

In [17]:
np.arccos(temp)

0.009299711826926139

In [8]:
jet_feats = ['j_pt','j_e','j_mass','j_phi','j_eta','j_theta']
for f in tqdm(jet_feats):
    eval(f).name = f
    par_df = par_df.join(f, on='j_index')

par_df.head()

,E,PX,PY,PZ,j_index,j_t,j_pt,j_e,j_mass,j_eta,j_phi,j_theta
0,474.071136,-250.347031,-223.651962,-334.738098,0,0,617.077006,869.288696,37.394853,-0.874543,-0.741334,-2.400258
1,103.236237,-48.866222,-56.790775,-71.025490,0,0,617.077006,869.288696,37.394853,-0.874543,-0.741334,-2.400258
2,105.255569,-55.415001,-49.968887,-74.236267,0,0,617.077006,869.288696,37.394853,-0.874543,-0.741334,-2.400258
3,40.176777,-21.760696,-18.717611,-28.112215,0,0,617.077006,869.288696,37.394853,-0.874543,-0.741334,-2.400258
4,22.428583,-11.835756,-10.374107,-15.979177,0,0,617.077006,869.288696,37.394853,-0.874543,-0.741334,-2.400258


In [9]:
# particle level features
j1_pt = (par_df['PX'].values**2+par_df['PY'].values**2)**.5
j1_phi = np.arcsin(par_df['PY'].values/par_df['j_pt'].values)
j1_eta = np.arcsinh(par_df['PZ'].values/par_df['j_pt'].values)

j1_theta = 2*np.arctan(np.exp(-par_df.j1_eta))
# j1_theta = np.arctan2(par_df['PY'].values, par_df['PX'].values)
# for i in tqdm(range(j1_theta.shape[0])):
#     if j1_theta[i] < 0:
#         j1_theta[i] += 2 * np.pi

# j1_etarot = j1_eta * np.cos(np.pi - j1_theta) - j1_phi * np.sin(np.pi - j1_theta)
# j1_phirot = j1_eta * np.sin(np.pi - j1_theta) + j1_phi * np.cos(np.pi - j1_theta)

j1_etarel = j1_eta - par_df['j_eta'].values
j1_phirel = j1_phi - par_df['j_phi'].values
j1_ptrel = j1_pt / par_df['j_pt'].values
j1_erel = par_df['E'].values / par_df['j_e'].values
j1_costhetarel = np.cos(j1_theta-par_df['j_theta'].values)

cIndex = np.array([],dtype='int8')
for i in tqdm(np.unique(par_df['j_index'])):
    new_cIndex = np.arange(par_df[par_df['j_index']==i].shape[0])
    cIndex = np.append(cIndex, new_cIndex)
_df['constituents_index'] = cIndex

par_feats = ['j1_pt','j1_phi','j1_eta','j1_etarot','j1_phirot','j1_etarel','j1_phirel','j1_ptrel','j1_erel','j1_costhetarel']
for f in tqdm(par_feats):
    par_df[f] = eval(f)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:39<00:00, 15.97s/it]


In [ ]:
par_df.drop(['E','PX','PY','PZ','j_e','j_eta','j_phi','j_theta'],axis=1, inplace=True)

NameError: name 'j_theta' is not defined